# Add Fossil Classification for a given Holding File

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

# Classify holdings file

In [4]:
holdings_path = "/Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports 2021Q3/holdings_for_cls.csv"
classify_holdings(holdings_path)


1.Preparing holding file

** Holdings file for classification **
/Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports 2021Q3/holdings_for_cls.csv
columns: Index(['שם המנפיק/שם נייר ערך', 'מספר ני"ע', 'מספר מנפיק', 'שווי',
       'שעור מנכסי אפיק ההשקעה', 'שעור מסך נכסי השקעה', 'holding_type',
       'זירת מסחר', 'תאריך רכישה', 'ערך נקוב', 'שער', 'שעור מערך נקוב מונפק',
       'ענף מסחר', 'SystemName', 'ParentCorpName', 'ReportPeriodDesc'],
      dtype='object')

Holding file ISIN col is: מספר ני"ע
number of ISINs: 1013 out of 2233 rows

Holding file Israel Corp col is: מספר מנפיק
number of Israel Corp Numbers: 1303 out of 2233 rows

2.Preparing mapping files


/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


מספר ני"עs with matching ISIN: 2033 out of total relevant rows: 1210
מספר תאגידs with matching מספר מנפיק: 1742 out of total relevant rows: 1303
מספר ני"עs with matching מספר מנפיק: 1743 out of total relevant rows: 1210
ISINs with matching מספר מנפיק: 1743 out of total relevant rows: 2033
ISINs with matching LEI: 668 out of total relevant rows: 2033


/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


מספר ני"עs with matching ISIN: 6727 out of total relevant rows: 1857
מספר תאגידs with matching מספר מנפיק: 1950 out of total relevant rows: 343
מספר ני"עs with matching מספר מנפיק: 1950 out of total relevant rows: 1857
ISINs with matching מספר מנפיק: 1950 out of total relevant rows: 6727
ISINs with matching LEI: 3848 out of total relevant rows: 6727

Writing results to prev with added issuer and LEI.csv

1. matching to previously classified by Israeli security number

previous is_fossil coverage
ISINs previously classified: 1153 out of total holdings: 2233

2. matching to previously classified by ISIN

previous is_fossil coverage
ISINs previously classified: 1867 out of total holdings: 2233

3. matching to previously classified by issuer number
issuers previously classified: 1298 out of total holdings: 2233

4. matching to previously classified by LEI
LEIs previously classified: 627 out of total holdings: 2233

5. matching to previously classified by מספר תאגיד
Israeli Corp Nums previo

/Users/urimarom/PycharmProjects/fossil_classification/fossil_classification.py:720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_col_not_null['is_fossil'] = grouped_by_prop_col['is_fossil'].transform(lambda x: x.fillna(x.mean()))



is_fossil coverage before propagation by מספר ני"ע:
0.00    1921
1.00     195
nan      117
Name: is_fossil, dtype: int64

is_fossil coverage after propagation by מספר ני"ע:
0.00    1921
1.00     195
nan      117
Name: is_fossil, dtype: int64

Propagating by LEI

is_fossil coverage before propagation by LEI:
0.00    1921
1.00     195
nan      117
Name: is_fossil, dtype: int64

is_fossil coverage after propagation by LEI:
0.00    1921
1.00     195
nan      117
Name: is_fossil, dtype: int64

Writing results to /Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports 2021Q3/holdings_for_cls with fossil classification.csv


# Manual Review
In a google spreadsheet or excel.
Download the fully classifed file into a csv, then use it in holding_cls_path to update prev_class (see below).

# Add classification results to prev_class

In [18]:
holdings_cls_path = "/Users/urimarom/PycharmProjects/fossil_classification/data/downloaded reports/company reports 2021Q3/migdal 2021Q3 with fossil classification.csv"
prev_class_path = "/Users/urimarom/PycharmProjects/fossil_classification/data_sources/prev_class.csv"
update_prev_class(holdings_cls_path, prev_class_path)

Adding classifications to prev_class, saving the previous version as /Users/urimarom/PycharmProjects/fossil_classification/data_sources/prev_class backup/prev_class 2021-12-02 18-53-05.csv
